# Testing a SAR-based ship classifier with different loss functions

In this notebook we explored six different Loss functions on SAR Data, we choose 3 classes of already available data and test the hypothesis of loss functions. The hypothesis is, Loss functions should be selected on the basis of dataset and task.

Libraries

In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from PIL import Image
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define data transformations

In [ ]:
# Define data transformations
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

Loading dataset

In [ ]:
def load_data(address):
  # Load Fusar dataset
  dataset = ImageFolder(root=address, transform=transform_train)

  # Create a dictionary of class names
  class_names = {i: classname for i, classname in enumerate(dataset.classes)}

  # Split dataset into train and test sets
  train_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_size
  train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

  train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,
                            num_workers=2,  # Experiment with different values as recommended above
                            # pin_memory=False, # if torch.cuda.is_available() else False,
                            persistent_workers=True)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                            num_workers=2,  # Experiment with different values as recommended above
                            # pin_memory=False, # if torch.cuda.is_available() else False,
                            persistent_workers=True)
  print("Top classes indices:", class_names)
  len(train_loader)*64, len(test_loader)*64, len(train_loader)*64+ len(test_loader)*64, dataset

  return train_loader, test_loader

Model Structure

In [ ]:
# Define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 28 * 28, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 3)  # 3 output classes
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model, input_size=(64, 3, 224, 224))  # Assuming 3-channel images

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [64, 3]                   --
├─Sequential: 1-1                        [64, 256, 28, 28]         --
│    └─Conv2d: 2-1                       [64, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [64, 64, 224, 224]        --
│    └─MaxPool2d: 2-3                    [64, 64, 112, 112]        --
│    └─Conv2d: 2-4                       [64, 128, 112, 112]       73,856
│    └─ReLU: 2-5                         [64, 128, 112, 112]       --
│    └─MaxPool2d: 2-6                    [64, 128, 56, 56]         --
│    └─Conv2d: 2-7                       [64, 256, 56, 56]         295,168
│    └─ReLU: 2-8                         [64, 256, 56, 56]         --
│    └─MaxPool2d: 2-9                    [64, 256, 28, 28]         --
├─Sequential: 1-2                        [64, 3]                   --
│    └─Linear: 2-10                      [64, 1024]                205,52

# FUSAR Dataset

Evaluation on different loss functions

In [ ]:
# importing the zipfile module
from zipfile import ZipFile

zip_file = "/content/fusar.zip"
path = "/content/fusar_data/"

# loading the temp.zip and creating a zip object
with ZipFile(zip_file, 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path=path)

In [ ]:
# Baseline loss function

loss_functions = {
    'CrossEntropyLoss': nn.CrossEntropyLoss(),
}
train_loader, test_loader = load_data('fusar_data')

Code for evaluation

In [ ]:
# Train the model with different loss functions
for loss_name, loss_func in loss_functions.items():
    print(f"Training with {loss_name} loss:")
    model = CNNModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    # Training loop
    for epoch in range(10):  # Train for 10 epochs
        model.train()
        print(epoch)
        for batch_idx, data in enumerate(train_loader):
            data, target = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(data)

            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 57 == 0:
                print(f'Epoch {epoch + 1}/{10}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy with {loss_name} loss function: {accuracy:.2f}%")
    print()

Comparing different functions

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, input, target):
        pt = torch.softmax(input, dim=1)
        pt = torch.argmax(pt, dim=1).float()
        ce_loss = nn.functional.cross_entropy(input, target, reduction='none')
        pt_with_loss = (1 - pt) ** self.gamma * ce_loss
        return pt_with_loss.mean()

loss_functions = {
    "BCEWithLogitsLoss":nn.BCEWithLogitsLoss(),
    'MSELoss': nn.MSELoss(),
    'L1Loss': nn.L1Loss(),
    'Focal_loss': FocalLoss(),
    'KLDiv': nn.KLDivLoss(reduction="batchmean", log_target=True),
    # Add more loss functions here if needed
}

In [ ]:
# Train the model with different loss functions
for loss_name, loss_func in loss_functions.items():
    print(f"Training with {loss_name} loss:")
    model = CNNModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    # Training loop
    for epoch in range(10):  # Train for 10 epochs
        model.train()
        print(epoch)
        for batch_idx, data in enumerate(train_loader):
            data, target = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(data)

            target = nn.functional.one_hot(target, num_classes=3).float()

            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 57 == 0:
                print(f'Epoch {epoch + 1}/{10}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy with {loss_name} loss function: {accuracy:.2f}%")
    print()

# Evaluating OpenSarShip


In [ ]:
# importing the zipfile module
from zipfile import ZipFile

zip_file = "/content/opensarship_png.zip"
path = "/content/opensarship_png/"

# loading the temp.zip and creating a zip object
with ZipFile(zip_file, 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path=path)


train_loader, test_loader = load_data('/content/opensarship_png')

In [ ]:
# Baseline loss function

loss_functions = {
    'CrossEntropyLoss': nn.CrossEntropyLoss(),
}

Top classes indices: {0: 'Cargo', 1: 'Fishing', 2: 'Tanker'}


In [ ]:
# Train the model with different loss functions
for loss_name, loss_func in loss_functions.items():
    print(f"Training with {loss_name} loss:")
    model = CNNModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    # Training loop
    for epoch in range(10):  # Train for 10 epochs
        model.train()
        print(epoch)
        for batch_idx, data in enumerate(train_loader):
            data, target = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(data)

            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 57 == 0:
                print(f'Epoch {epoch + 1}/{10}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy with {loss_name} loss function: {accuracy:.2f}%")
    print()

In [ ]:
# Train the model with different loss functions
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, input, target):
        pt = torch.softmax(input, dim=1)
        pt = torch.argmax(pt, dim=1).float()
        ce_loss = nn.functional.cross_entropy(input, target, reduction='none')
        pt_with_loss = (1 - pt) ** self.gamma * ce_loss
        return pt_with_loss.mean()

loss_functions = {
    "BCEWithLogitsLoss":nn.BCEWithLogitsLoss(),
    'MSELoss': nn.MSELoss(),
    'L1Loss': nn.L1Loss(),
    'Focal_loss': FocalLoss(),
    'KLDiv': nn.KLDivLoss(reduction="batchmean", log_target=True),
    # Add more loss functions here if needed
}

for loss_name, loss_func in loss_functions.items():
    print(f"Training with {loss_name} loss:")
    model = CNNModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    # Training loop
    for epoch in range(10):  # Train for 10 epochs
        model.train()
        print(epoch)
        for batch_idx, data in enumerate(train_loader):
            data, target = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            output = model(data)

            target = nn.functional.one_hot(target, num_classes=3).float()

            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 57 == 0:
                print(f'Epoch {epoch + 1}/{10}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}')

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy with {loss_name} loss function: {accuracy:.2f}%")
    print()